In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_consult').getOrCreate()

In [2]:
data = spark.read.csv('/FileStore/tables/dog_food.csv',
                     inferSchema=True,
                     header = True)

In [3]:
data.head(2)

Out[ 4 ]: 
[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)]

In [4]:
data.columns

Out[ 5 ]: ['A', 'B', 'C', 'D', 'Spoiled']

In [5]:
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [6]:
data.describe()

Out[ 7 ]: DataFrame[summary: string, A: string, B: string, C: string, D: string, Spoiled: string]

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols = ['A', 'B', 'C', 'D'], outputCol = 'features')

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.classification import RandomForestClassifier

In [11]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol = 'features')

In [12]:
output.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)
-- features: vector (nullable = true)

In [13]:
final_data = output.select(['features', 'Spoiled'])

In [14]:
final_data.show()

+-------------------+-------+
 features|Spoiled|
+-------------------+-------+
 [4.0,2.0,12.0,3.0]| 1.0|
 [5.0,6.0,12.0,7.0]| 1.0|
 [6.0,2.0,13.0,6.0]| 1.0|
 [4.0,2.0,12.0,1.0]| 1.0|
 [4.0,2.0,12.0,3.0]| 1.0|
[10.0,3.0,13.0,9.0]| 1.0|
 [8.0,5.0,14.0,5.0]| 1.0|
 [5.0,8.0,12.0,8.0]| 1.0|
 [6.0,5.0,12.0,9.0]| 1.0|
 [3.0,3.0,12.0,1.0]| 1.0|
 [9.0,8.0,11.0,3.0]| 1.0|
[1.0,10.0,12.0,3.0]| 1.0|
[1.0,5.0,13.0,10.0]| 1.0|
[2.0,10.0,12.0,6.0]| 1.0|
[1.0,10.0,11.0,4.0]| 1.0|
 [5.0,3.0,12.0,2.0]| 1.0|
 [4.0,9.0,11.0,8.0]| 1.0|
 [5.0,1.0,11.0,1.0]| 1.0|
[4.0,9.0,12.0,10.0]| 1.0|
 [5.0,8.0,10.0,9.0]| 1.0|
+-------------------+-------+
only showing top 20 rows

In [15]:
rfc_model = rfc.fit(final_data)

In [16]:
final_data.head(1)

Out[ 29 ]: [Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [17]:
rfc_model.featureImportances

Out[ 30 ]: SparseVector(4, {0: 0.0179, 1: 0.0312, 2: 0.9235, 3: 0.0275})